In [1]:
from datasets import load_dataset
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
rag_dataset = load_dataset("neural-bridge/rag-dataset-12000")

/Users/codym/CAP/aisk_forked/ai-starter-kit/genai_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rag_dataset

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 9600
    })
    test: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 2400
    })
})

In [3]:
len(rag_dataset["train"]["context"][0])
subset = rag_dataset["train"]["context"][:100]

In [6]:
text_path = "/Users/codym/CAP/aisk_forked/ai-starter-kit/complex_rag/data/"
text_file = "subset.txt"

full_path = text_path + text_file
if not os.path.exists(text_path):
        os.makedirs(text_path)

print(full_path)

with open(full_path, "w") as f:
    [f.write('\n\n'.join([l for l in subset]))]

/Users/codym/CAP/aisk_forked/ai-starter-kit/complex_rag/data/subset.txt


In [7]:
from langchain_community.embeddings.sambanova import SambaStudioEmbeddings
from langchain.vectorstores import Chroma

os.environ["SAMBASTUDIO_EMBEDDINGS_BASE_URL"] = "https://sjc3-demo2.sambanova.net"
os.environ["SAMBASTUDIO_EMBEDDINGS_PROJECT_ID"] = "2eeb2d6c-525a-481d-8019-9032f66e5339"
os.environ["SAMBASTUDIO_EMBEDDINGS_ENDPOINT_ID"] = "3c6ff0c2-94a2-424c-9f21-854506f60c58"
os.environ["SAMBASTUDIO_EMBEDDINGS_API_KEY"] = "661a73b2-7661-424c-89ec-43c314e94a78"

embeddings = SambaStudioEmbeddings()

with open(full_path, "r") as f:
    txts = f.read()

# for l in [1000]:

text_splitter = RecursiveCharacterTextSplitter(
    
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents([txts])

# To save
vectorstore = Chroma.from_documents(documents=texts, 
                                    embedding=embeddings, 
                                    persist_directory="/Users/codym/CAP/aisk_forked/ai-starter-kit/complex_rag/data/rag_dataset_12000_train_100samples_1000_100.chromadb"
        )

# To load
# vectorstore = Chroma(persist_directory="/Users/codym/CAP/ekr_lg/langgraph_ekr/enterprise_knowledge_retriever/data/rag_dataset_12000_train_1000_100.chromadb", embedding_function=embeddings)

In [ ]:
# from langchain_community.embeddings.sambanova import SambaStudioEmbeddings
# from langchain.vectorstores import Chroma

# os.environ["SAMBASTUDIO_EMBEDDINGS_BASE_URL"] = "https://sjc3-demo2.sambanova.net"
# os.environ["SAMBASTUDIO_EMBEDDINGS_PROJECT_ID"] = "2eeb2d6c-525a-481d-8019-9032f66e5339"
# os.environ["SAMBASTUDIO_EMBEDDINGS_ENDPOINT_ID"] = "3c6ff0c2-94a2-424c-9f21-854506f60c58"
# os.environ["SAMBASTUDIO_EMBEDDINGS_API_KEY"] = "661a73b2-7661-424c-89ec-43c314e94a78"

# embeddings = SambaStudioEmbeddings()
# # To save
# # vectorstore = Chroma.from_documents(documents=texts, 
# #                                     embedding=embeddings, 
# #                                     persist_directory="/Users/codym/CAP/ekr_lg/langgraph_ekr/enterprise_knowledge_retriever/data/rag_dataset_12000_train_1000_100.chromadb"
# #         )

# # To load
# # vectorstore = Chroma(persist_directory="/Users/codym/CAP/ekr_lg/langgraph_ekr/enterprise_knowledge_retriever/data/rag_dataset_12000_train_1000_100.chromadb", embedding_function=embeddings)

In [ ]:
embeds = embeddings.embed_documents(rag_dataset["train"]["context"])

In [ ]:
len(embeds)

In [ ]:
rag_dataset["train"]["question"][0]

In [ ]:
vectorstore.similarity_search(rag_dataset["train"]["question"][0])

In [ ]:
reducer = umap.UMAP()

In [ ]:
umap_embeddings = reducer.fit_transform(embeds)
umap_embeddings.shape

In [ ]:
umap_embeddings[:,1]

In [ ]:
import textwrap
def wrap_text(text, width=30):
    return '<br>'.join(textwrap.fill(text, width).split('\n'))

wrapped_text = []
for i in range(len(rag_dataset["train"]["context"])):
    wrapped_text.append(wrap_text(rag_dataset["train"]["context"][i]))

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(umap_embeddings[:,0], umap_embeddings[:,1], hover_data={"Question": rag_dataset["train"]["question"], "Context": wrapped_text, "Answer": rag_dataset["train"]["answer"]}, width=1600, height=800, opacity=0.6)

fig.update_layout(hoverlabel=dict(font_size=12, font_family="Century Gothic"))

fig.show()

In [ ]:
fig.write_html("/Users/codym/CAP/ekr_lg/langgraph_ekr/images/train_umap.html")